In [17]:
import re
import json
import time 

import requests
import urllib.request
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [14]:
def web_content(link):
    """¨función que obtiene el nombre, latitud y longitud para cada uno link de estado que se le introduce."""
    datos = []
    html = requests.get(link).content
    soup = BeautifulSoup(html,'html')
    nombre = soup.h1.string
    datos.append(nombre)
    latitud = soup.select('span[class="latitude"]')
    latitud = re.findall('\d+\.\d+',latitud[1].text)
    datos.append(latitud[0])
    longitud = soup.select('span[class="longitude"]')
    datos.append(longitud[1].text.strip())
    return datos

def getOffices(state,location):
    """Función que obtiene las sucursales en un radio establecido de 50km mediante un texto de búsqueda, una latitud y 
    longitud."""
    #Indice el formato de salida para el request
    output = 'json'
    input_text = 'bbva+'+state.replace(" ","+").replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
    
    #Hace el request a la API
    url=f'https://maps.googleapis.com/maps/api/place/textsearch/{output}?query={input_text}&location={location}&radius=50000&key={YOUR_API_KEY}'
    request = urllib.request.urlopen(url)
    data = json.load(request)
    
    #Crea un DataFrame con las sucursales que encontró, selecciona las columnas de interés y las renombra.
    data_norm = pd.json_normalize(data['results'])    
    data_norm = data_norm[['business_status','formatted_address','name','place_id','rating','types','geometry.location.lat', 'geometry.location.lng']]
    data_norm = data_norm.rename(columns={'geometry.location.lat':'latitude','geometry.location.lng':'longitude'})
    data_norm['state'] = state 
    return data_norm

def getPlaces(location,radius,place_id_suc,type_place):
    """Función que obtiene distintos tipos de establecimientos comerciales en un radio seleccionado de acuerdo a la latitud y 
    longitud de una sucursal ."""
    #Indice el formato de salida para el request
    output = 'json'
    data_base = pd.DataFrame(columns=['place_id','name','rating','formatted_address','latitude','longitude','types','place_id_sucursal'])
    url = f'https://maps.googleapis.com/maps/api/place/textsearch/{output}?location={location}&radius={radius}&languages=es-419&type={type_place}&key={YOUR_API_KEY}'
    #time.sleep(3)
    try:
        request = urllib.request.urlopen(url,timeout=5)
        data = json.load(request)
        data_norm = pd.json_normalize(data['results'])
        data_norm = data_norm[['place_id','name','rating','formatted_address','geometry.location.lat', 'geometry.location.lng']]
        data_norm = data_norm.rename(columns={'geometry.location.lat':'latitude','geometry.location.lng':'longitude'})
        data_norm['types'] = type_place
        data_norm['place_id_sucursal'] = place_id_suc
        return data_norm
    except:
        return data_base

In [3]:
#Autoriza la API de Google Maps
with open('../../client.txt') as f:
    YOUR_API_KEY = f.readlines()
    YOUR_API_KEY = YOUR_API_KEY[0].split()
    YOUR_API_KEY = YOUR_API_KEY[0]

Extracción de la latitud y longitud para cada Entidad Federativa mediante web scraping.

In [4]:
#Exporta la información de los estados
url = 'https://es.wikipedia.org/wiki/Organizaci%C3%B3n_territorial_de_M%C3%A9xico'
html = requests.get(url).content
soup = BeautifulSoup(html,'html')

#Crea una lista de enlaces de los estados para obtener su latitud y longitud
estados = soup.select('td>a[class="mw-redirect"]:first-child')
link_estados = ['https://es.wikipedia.org/' + estados[i]['href'] for i in range(len(estados)-4)]

#Limpia y genera la lista definitiva
link_estados = list(dict.fromkeys(link_estados))
link_estados.pop(1)
link_estados.pop(19)
link_estados.pop(20)
link_estados.append('https://es.wikipedia.org/wiki/Ciudad_de_M%C3%A9xico')
link_estados.append('https://es.wikipedia.org/wiki/Estado_de_Guerrero')
link_estados.append('https://es.wikipedia.org/wiki/Estado_de_Hidalgo')
link_estados.append('https://es.wikipedia.org/wiki/Estado_de_M%C3%A9xico')
link_estados.append('https://es.wikipedia.org/wiki/Quer%C3%A9taro')

#Verificar que están los 32 estados
len(link_estados)

32

In [5]:
#Entra a cada una de las páginas por estado para obtener nombre, latitud y longitud
lat_lon_estados = []
for link in link_estados:
    lat_lon_estados.append(web_content(link))

In [22]:
#lat_lon_estados

 Solicitud de sucursales por Entidad Federativa de la API de GCP.

In [7]:
#Genera un dataFrame donde se va a almacenar la información de las sucursales por estado.
sucursales = pd.DataFrame(columns=['business_status','formatted_address','name','place_id','rating','types','latitude', 'longitude','state'])

#Busca para cada uno de los estados, las 20 primeras sucursales que encuentra con los parámetros enviados (topado por la API).
for i in range(len(lat_lon_estados)):
    state = lat_lon_estados[i][0]
    location = lat_lon_estados[i][1]+','+lat_lon_estados[i][2]
    sucursales = pd.concat([sucursales,getOffices(state,location)])

#Reinicia el index para poder hacer uso de cada una de las filas    
sucursales = sucursales.reset_index(drop=True)

In [20]:
#La API proporciona hasta 20 resultados.
#sucursales['state'].value_counts()
#sucursales

Solicitud de establecimientos por tipo y distancia para cada una de las sucursales, de la API de GCP.

In [9]:
#Determina el radio en el que se van a buscar los establecimietnos con respecto a la sucursal
radius = 1000
types = ['airport','car_dealer','hospital','restaurant','shopping_mall','stadium','supermarket']
places = pd.DataFrame(columns=['place_id','name','rating','formatted_address','latitude','longitude','types','place_id_sucursal'])
for type_place in types:    
    for i in range(len(sucursales)):
        location = str(sucursales.at[i,'latitude'])+','+str(sucursales.at[i,'longitude'])
        place_id_suc = sucursales.at[i,'place_id']
        places = pd.concat([places,getPlaces(location,radius,place_id_suc,type_place)])
    time.sleep(10)
    
#Reinicia el index para poder hacer uso de cada una de las filas    
places = places.reset_index(drop=True)

In [21]:
#places

Tratamiento de datos.

In [13]:
#Generamos variables dummies
places_resume = pd.get_dummies(places['types'])
places_resume.head()

,airport,car_dealer,hospital,restaurant,shopping_mall,stadium,supermarket
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [16]:
#Junta el dataframe original + dummies
places_resume = pd.concat([places,places_resume],axis=1)
places_resume.head()

,place_id,name,rating,formatted_address,latitude,longitude,types,place_id_sucursal,airport,car_dealer,hospital,restaurant,shopping_mall,stadium,supermarket
0,ChIJKdGzCceTKYQRo0IzX9lQ6g8,International Airport of Aguascalientes,4,"Carr. Panamericana Km. 22, 20340 Buenavista de...",21.701298,-102.316228,airport,ChIJkySim3PuKYQR6VtatBtbYKY,1,0,0,0,0,0,0
1,ChIJIY2O9MOTKYQRyjeYqI2owGA,Volaris Aguascalientes,3.7,"Carr. Panamericana Km. 22, 20340 Aguascaliente...",21.701481,-102.314160,airport,ChIJkySim3PuKYQR6VtatBtbYKY,1,0,0,0,0,0,0
2,ChIJaQAk1apNL4QRnW99Ljn8FEw,Miguel Hidalgo y Costilla International Airport,4.3,"Carr. Guadalajara - Chapala Km 17.5, 45659 Jal.",20.525956,-103.307625,airport,ChIJkySim3PuKYQR6VtatBtbYKY,1,0,0,0,0,0,0
3,ChIJ3RBtpjIegIYRUTAg7gjjdVM,San Luis Potosí International Airport,4,"Carr. a Matehuala Km 9.5, 78341 San Luis, S.L.P.",22.256886,-100.934151,airport,ChIJkySim3PuKYQR6VtatBtbYKY,1,0,0,0,0,0,0
4,ChIJK21u06TtKYQRTaDGg3BiZqs,Helipuerto Bicentenario Municipal,5,"Carolina Villanueva de García S/N, Cd Industri...",21.835724,-102.285848,airport,ChIJkySim3PuKYQR6VtatBtbYKY,1,0,0,0,0,0,0


In [25]:
#Suma cada una de las categorías por id de sucursal
types = places_resume.groupby('place_id_sucursal',as_index=False).agg({'car_dealer':np.sum,'hospital':np.sum,'restaurant':np.sum,'shopping_mall':np.sum,'stadium':np.sum,'supermarket':np.sum})
types = types.rename(columns={'place_id_sucursal':'place_id'})
#types

,place_id,car_dealer,hospital,restaurant,shopping_mall,stadium,supermarket
0,ChIJ--C2IvjAz4URlQ-736Zk9gE,20,20,20,20,20,20
1,ChIJ--VUmhLY7YURtEUdOTSr2S0,20,20,20,20,20,20
2,ChIJ-TEZZgv_0YUR85S8DVucxFo,40,40,40,40,40,40
3,ChIJ-V0fYNKYKoQRdNB8JMqs0Lk,20,20,20,20,12,20
4,ChIJ-VCx443x7YUReGM0do_mxww,20,20,20,9,20,19
...,...,...,...,...,...,...,...
521,ChIJzSyxthYt0oURFG-fKkkfTxw,20,20,20,20,20,18
522,ChIJzTmuHxZF7YUROMAH4fik1uQ,20,20,20,20,20,16
523,ChIJzb03B7Vc6oYR0AD8wq3rCZQ,20,20,20,20,20,20
524,ChIJze1N-lpxVo8Rt4Fwk2bJIis,20,20,20,20,14,20


In [26]:
details = pd.merge(sucursales, types, on='place_id')
details.head()

,business_status,formatted_address,name,place_id,rating,types,latitude,longitude,state,car_dealer,hospital,restaurant,shopping_mall,stadium,supermarket
0,OPERATIONAL,"Av LIC.Adolfo López M.Ote 1001, San Luís, 2025...",BBVA Aguascalientes Quijote,ChIJkySim3PuKYQR6VtatBtbYKY,3.2,"[bank, atm, finance, point_of_interest, establ...",21.877871,-102.273797,Aguascalientes,20,20,20,20,20,20
1,OPERATIONAL,"Av, 5 de Mayo 112, 20000 Aguascalientes, Ags.,...",BBVA Bancomer AGS Office Center,ChIJQeb3zXzuKYQRL0tIOCJIxj4,3.8,"[bank, atm, finance, point_of_interest, establ...",21.881528,-102.296980,Aguascalientes,20,20,20,20,20,18
2,OPERATIONAL,"Av Aguascalientes Sur 603, Cd Industrial, Desa...",BBVA Aguascalientes,ChIJkfpGScXtKYQR3SfdWyirAw0,3.4,"[bank, atm, finance, point_of_interest, establ...",21.858630,-102.287521,Aguascalientes,20,20,20,20,20,18
3,OPERATIONAL,"Avenida Universidad 117, Bosques del Prado Sur...",BBVA Bosques,ChIJuSceueLuKYQRGVX2q-UOtJ8,3.0,"[bank, atm, finance, point_of_interest, establ...",21.914717,-102.312611,Aguascalientes,20,20,20,20,20,20
4,OPERATIONAL,"Agropecuario, Av Aguascalientes Nte 1698, 2013...",BBVA Aguascalientes Pulgas Pandas,ChIJIxqo6FTuKYQR_nCSHsOk7Rc,3.4,"[bank, atm, finance, point_of_interest, establ...",21.915216,-102.296958,Aguascalientes,20,20,20,20,20,20


Generación de archivos finales.

In [28]:
places_file = details.to_csv('Outputs/sucursales.csv',index=False)

In [29]:
estados = pd.DataFrame(lat_lon_estados,columns=['Estado','Latitud','Longitud'])
estados_file = estados.to_csv('Outputs/estados.csv',index=False)